# concurrent并发包

In [2]:
import concurrent
help(concurrent)

Help on package concurrent:

NAME
    concurrent - # This directory is a Python package.

MODULE REFERENCE
    https://docs.python.org/3.6/library/concurrent
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

PACKAGE CONTENTS
    futures (package)

FILE
    /usr/lib/python3.6/concurrent/__init__.py




In [6]:
help(concurrent.futures)

Help on package concurrent.futures in concurrent:

NAME
    concurrent.futures - Execute computations asynchronously using threads or processes.

MODULE REFERENCE
    https://docs.python.org/3.6/library/concurrent.futures
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

PACKAGE CONTENTS
    _base
    process
    thread

DATA
    ALL_COMPLETED = 'ALL_COMPLETED'
    FIRST_COMPLETED = 'FIRST_COMPLETED'
    FIRST_EXCEPTION = 'FIRST_EXCEPTION'

AUTHOR
    Brian Quinlan (brian@sweetapp.com)

FILE
    /usr/lib/python3.6/concurrent/futures/__init__.py




In [9]:
#help(concurrent.futures.process)
# 以thread为例，进程差不多
help(concurrent.futures.thread)

Help on module concurrent.futures.thread in concurrent.futures:

NAME
    concurrent.futures.thread - Implements ThreadPoolExecutor.

MODULE REFERENCE
    https://docs.python.org/3.6/library/concurrent.futures.thread
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

CLASSES
    concurrent.futures._base.Executor(builtins.object)
        ThreadPoolExecutor
    
    class ThreadPoolExecutor(concurrent.futures._base.Executor)
     |  This is an abstract base class for concrete asynchronous executors.
     |  
     |  Method resolution order:
     |      ThreadPoolExecutor
     |      concurrent.futures._base.Executor
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, max_

# 1.Executor

Executor是一个抽象类，它不能被直接使用。它为具体的异步执行定义了一些基本的方法。 
ThreadPoolExecutor和ProcessPoolExecutor继承了Executor，分别被用来创建线程池和进程池的代码。

In [11]:
help(concurrent.futures._base.Executor)

Help on class Executor in module concurrent.futures._base:

class Executor(builtins.object)
 |  This is an abstract base class for concrete asynchronous executors.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |  
 |  map(self, fn, *iterables, timeout=None, chunksize=1)
 |      Returns an iterator equivalent to map(fn, iter).
 |      
 |      Args:
 |          fn: A callable that will take as many arguments as there are
 |              passed iterables.
 |          timeout: The maximum number of seconds to wait. If None, then there
 |              is no limit on the wait time.
 |          chunksize: The size of the chunks the iterable will be broken into
 |              before being passed to a child process. This argument is only
 |              used by ProcessPoolExecutor; it is ignored by
 |              ThreadPoolExecutor.
 |      
 |      Returns:
 |          An iterator equivalent to: map(func, *iterables) but the call

# 2.submit()

Executor中定义了submit()方法，这个方法的作用是提交一个可执行的回调task，并返回一个future实例。future对象代表的就是给定的调用。 
通过下面的例子来理解submit对线程池/进程池的操作:

In [13]:
from concurrent.futures import ThreadPoolExecutor
import time

def get_future(msg):
    time.sleep(2)
    return msg

# 创建线程池
pool = ThreadPoolExecutor(max_workers=2)

f1 = pool.submit(get_future,'jimo')
f2 = pool.submit(get_future,'lonely')

print(f1.done())
time.sleep(2)
print(f2.done())

print(f1.result())
print(f2.result())

False
True
jimo
lonely


# 3.map()

除了submit，Exectuor还为我们提供了map方法，这个方法返回一个map(func, *iterables)迭代器，迭代器中的回调执行返回的结果有序的。可以通过下面的例子来理解：

In [14]:
from concurrent.futures import ThreadPoolExecutor
import requests

URLS = ['http://www.baidu.com','http://www.sougou.com','http://www.taobao.com']

def task(url):
    return requests.get(url)

pool = ThreadPoolExecutor(max_workers=3)
result = pool.map(task,URLS)

for r in result :
    print(r.url,len(r.content))

http://www.baidu.com/ 2381
http://www.sougou.com/ 69853
https://www.taobao.com/ 123851


# 4.Future

In [15]:
# coding: utf-8
from concurrent.futures import ThreadPoolExecutor as Pool
from concurrent.futures import as_completed
import requests

URLS = ['http://qq.com', 'http://sina.com', 'http://www.baidu.com', ]


def task(url, timeout=10):
    return requests.get(url, timeout=timeout)


with Pool(max_workers=3) as executor:
    future_tasks = [executor.submit(task, url) for url in URLS]

    for f in future_tasks:
        if f.running():
            print('%s is running' % str(f))

    for f in as_completed(future_tasks):
        try:
            ret = f.done()
            if ret:
                f_ret = f.result()
                print('%s, done, result: %s, %s' % (str(f), f_ret.url, len(f_ret.content)))
        except Exception as e:
            f.cancel()
            print(str(e))

<Future at 0x7f4347eaecf8 state=running> is running
<Future at 0x7f4346396048 state=running> is running
<Future at 0x7f4346396ba8 state=running> is running
<Future at 0x7f4346396ba8 state=finished returned Response>, done, result: http://www.baidu.com/, 2381
<Future at 0x7f4347eaecf8 state=finished returned Response>, done, result: http://www.qq.com/, 251165
<Future at 0x7f4346396048 state=finished returned Response>, done, result: http://www.sina.com.cn/, 600030
